# Data Preparation

## Import Libraries

In [1]:
from posix import getcwd

from jupyterlab.commands import get_workspaces_dir
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install geopandas
!pip install wordcloud
!pip install mapclassify
!pip install mgwr
!pip install statsmodels


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

## Load and Merge Datasets

There are 5 datasets that we will load and merge into one. Our main dataset is Austin housing price, which has most of the features needed for the model prediction. Besides, there is a dataset contain population and density for every zip code in the USA, a dataset contains income per zip code for Austin, mortage rate for Austin and median list price per square feet. 

### The 1st dataset has information about population and density for every zip code in the US

In [36]:
zip = pd.read_csv("uszips.csv")

In [39]:
#There are more than 30 000 cities in this data set, hence, we only need data for Austin, so in the next step, we will filter it
zip['city'].nunique

<bound method IndexOpsMixin.nunique of 0           Adjuntas
1             Aguada
2          Aguadilla
3            Maricao
4             Anasco
            ...     
33778          Hyder
33779        Klawock
33780     Metlakatla
33781    Point Baker
33782       Wrangell
Name: city, Length: 33783, dtype: object>

In [41]:
zip[zip['city'] == 'Austin'].count()

zip                 51
lat                 51
lng                 51
city                51
state_id            51
state_name          51
zcta                51
parent_zcta          0
population          51
density             51
county_fips         51
county_name         51
county_weights      51
county_names_all    51
county_fips_all     51
imprecise           51
military            51
timezone            51
dtype: int64

In [85]:
#reduce the data set to include only zip code for Austin and its population & density
zip_austin = zip.loc[zip['city'].str.lower() == 'austin', ['zip','population', 'density']]

In [74]:
zip_austin

,population,density
5192,"1,196",2
13838,"1,203",21
15678,"6,647",61
19143,"29,438",60
25219,"9,945",84
27913,"11,114","2,845"
27914,"26,495","2,044"
27915,"22,484","1,544"
27916,"50,479","2,224"
27917,"32,937","6,143"


In [45]:
#remove the row which has density = 0, this might due to lack of data
zip_austin = zip_austin[zip_austin['density'] != 0]

### Another data set with Austin income by zip code is loaded

In [149]:
income = pd.read_csv('Income By Zipcode.csv', sep =';')
income['ZIP'].nunique()

,ZIP,Income
0,78617,78019
1,78652,118750
2,78701,154867
3,78702,101304
4,78703,144637


In [150]:
income.head()

,ZIP,Income
0,78617,78019
1,78652,118750
2,78701,154867
3,78702,101304
4,78703,144637


### The main dataset about Austin housing 

In [48]:
data = pd.read_csv('austinHousingData.csv')
data = data[data['city'] == 'austin']

In [151]:
data.head()

,zpid,city,streetAddress,zipcode,description,latitude,longitude,propertyTaxRate,garageSpaces,hasAssociation,...,MedianStudentsPerTeacher,numOfBathrooms,numOfBedrooms,numOfStories,homeImage,ZIP,Income,zip,population,density
0,116080992,austin,10505 Cannon Mark Way,78717,*PLEASE SEE THE WALKTHROUGH VIRTUAL TOUR*This ...,30,-98,2,0,True,...,14,6,5,2,116080992_83c089e97b4e96ecb01c3555fa8252d0-p_f...,"78,717","140,483","78,717","36,085","1,068"
1,88013985,austin,15524 Staked Plains Loop,78717,Rare opportunity in this high demand area of A...,30,-98,2,2,True,...,16,4,5,2,88013985_6d4b35d2a232c42fcc863317f22401c7-p_f.jpg,"78,717","140,483","78,717","36,085","1,068"
2,70344594,austin,15217 Sabal Palm Rd,78724,Charming Home boasting 3 bedrooms and 2 baths....,30,-98,2,2,False,...,13,2,3,1,70344594_3e29ab1c54a4a8ca80360911b9ac7434-p_f.jpg,"78,724","76,906","78,724","29,326",454
3,185217484,austin,14004 Chisos Trl,78717,"COVETED, SPACIOUS 3 bed + OFFICE, 1story MIL P...",30,-98,2,0,True,...,16,2,3,1,185217484_1f06634bc9d5424bc0f094f68ca72fe4-p_f...,"78,717","140,483","78,717","36,085","1,068"
5,116080981,austin,14405 Laurinburg Dr,78717,Popular Bingham Floor plan in sought after Nor...,30,-98,2,2,True,...,14,5,4,2,116080981_b15ecd54b82b93b6af1408beb22e069f-p_f...,"78,717","140,483","78,717","36,085","1,068"


In [65]:
data['zipcode'].nunique()

45

In [90]:
#merge population, density and income with zip code to the main dataset
data = data.merge(income, left_on='zipcode', right_on='ZIP', how='left')\
           .merge(zip_austin, left_on='zipcode', right_on='zip', how='left')

In [94]:
#in merging process, there are some missing value for certain area, we will investigate and fill the missing value
print(data.isnull().sum())

zpid                            0
city                            0
streetAddress                   0
zipcode                         0
description                     2
latitude                        0
longitude                       0
propertyTaxRate                 0
garageSpaces                    0
hasAssociation                  0
hasCooling                      0
hasGarage                       0
hasHeating                      0
hasSpa                          0
hasView                         0
homeType                        0
parkingSpaces                   0
yearBuilt                       0
latestPrice                     0
numPriceChanges                 0
latest_saledate                 0
latest_salemonth                0
latest_saleyear                 0
latestPriceSource               0
numOfPhotos                     0
numOfAccessibilityFeatures      0
numOfAppliances                 0
numOfParkingFeatures            0
numOfPatioAndPorchFeatures      0
numOfSecurityF

In [97]:
#checking for income data missing 
missing_income = data[data['ZIP'].isnull() | data['Income'].isnull()]
missing_income['zipcode'].unique()

array([78734, 78737, 78736, 78738, 78653])

As of now, the income data for 5 zip code is missing. We are taking Median Household Income data from https://www.incomebyzipcode.com/texas to fill the missing value:

For 78734: 129,116 <br>
For 78737: 164,066 <br>
For 78736: 100,761 <br>
For 78738: 169,911 <br>
For 78653: 92,799 <br>

In [101]:
fill_income = {
    78734: 129116,
    78737: 164066,
    78736: 100761,
    78738: 169911,
    78653: 92799}

for zip_code, median_income in fill_income.items():
    mask = (data['zipcode'] == zip_code) & (data['Income'].isnull())
    data.loc[mask, 'Income'] = median_income

filled_income = data.loc[data['zipcode'].isin(fill_income.keys()), ['zipcode', 'Income']]
print(filled_income)

       zipcode  Income
355      78734 129,116
364      78734 129,116
378      78734 129,116
431      78737 164,066
442      78737 164,066
...        ...     ...
12842    78736 100,761
13803    78734 129,116
13810    78737 164,066
13812    78736 100,761
13814    78737 164,066

[786 rows x 2 columns]


In [102]:
#check rows with misisng population data
missing_pop = data[data['zip'].isnull() | data['population'].isnull()]
missing_pop['zipcode'].unique()

array([78652, 78653])

Also from https://www.incomebyzipcode.com/texas/ there is information about population and density

For 78652: 

Estimated Population: Approximately 7,500 residents <br>
Population Density: Roughly 1,300 people per square mile

For 78653:

Estimated Population: Approximately 8,200 residents <br>
Population Density: Roughly 1,400 people per square mile

In [103]:
# Define a dictionary mapping the zip codes to their population and density estimates.
fill_pop = {
    78652: {'population': 7500, 'density': 1300},
    78653: {'population': 8200, 'density': 1400}}

for zip_code, values in fill_pop.items():
    mask = data['zipcode'] == zip_code
    data.loc[mask, 'population'] = values['population']
    data.loc[mask, 'density'] = values['density']

filled_pop = data.loc[data['zipcode'].isin(fill_pop.keys()), ['zipcode', 'population', 'density']]
print(filled_pop)

       zipcode  population  density
750      78652       7,500    1,300
10686    78653       8,200    1,400


In [119]:
print(data.columns)

Index(['zpid', 'city', 'streetAddress', 'zipcode', 'description', 'latitude',
       'longitude', 'propertyTaxRate', 'garageSpaces', 'hasAssociation',
       'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa', 'hasView',
       'homeType', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'latest_salemonth',
       'latest_saleyear', 'latestPriceSource', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'homeImage', 'ZIP', 'Income', 'zip',
       'population', 'density'],

In [124]:
#there are certain columns which are not meaningful so we will drop them
cols_to_drop = ['zpid', 'city', 'streetAddress', 'zipcode', 'description','homeImage','homeType', 'latestPriceSource', 'ZIP', 'zip', 'latest_salemonth','latest_saleyear']

In [125]:
df = data.drop(columns=cols_to_drop, errors='ignore')

In [126]:
#check if there is any null value left after remove those columns
df.isnull().sum()

latitude                      0
longitude                     0
propertyTaxRate               0
garageSpaces                  0
hasAssociation                0
hasCooling                    0
hasGarage                     0
hasHeating                    0
hasSpa                        0
hasView                       0
parkingSpaces                 0
yearBuilt                     0
latestPrice                   0
numPriceChanges               0
latest_saledate               0
numOfPhotos                   0
numOfAccessibilityFeatures    0
numOfAppliances               0
numOfParkingFeatures          0
numOfPatioAndPorchFeatures    0
numOfSecurityFeatures         0
numOfWaterfrontFeatures       0
numOfWindowFeatures           0
numOfCommunityFeatures        0
lotSizeSqFt                   0
livingAreaSqFt                0
numOfPrimarySchools           0
numOfElementarySchools        0
numOfMiddleSchools            0
numOfHighSchools              0
avgSchoolDistance             0
avgSchoo

In [127]:
#check if there is any duplicate value
df.duplicated().sum()

np.int64(0)

At this point, we have dataframe 'df' with all the features

### For the next part, we will incorporate macroeconomic factors, which are mortage rate and median list price per square feet from the other 2 datasets

In [130]:
MTG = pd.read_csv('MORTGAGE30US.csv')
MPPSQ = pd.read_csv('MEDLISPRIPERSQUFEE12420.csv')

In [131]:
def parse_obs_date(date_str):
    d = str(date_str).strip().split()[0]
    if "/" in d:
        return pd.to_datetime(d, format='%d/%m/%Y', errors='raise')
    else:
        return pd.to_datetime(d, errors='raise')

MTG['observation_date'] = MTG['observation_date'].apply(parse_obs_date)
MPPSQ['observation_date'] = MPPSQ['observation_date'].apply(parse_obs_date)
MTG['MORTGAGE30US'].dtypes

dtype('float64')

In [ ]:
df['latest_saledate'] = pd.to_datetime(df['latest_saledate'], errors='coerce')
df = df.sort_values('latest_saledate')

In [140]:
#merge mortgate rate
MTG['observation_date'] = pd.to_datetime(MTG['observation_date'], format='%d/%m/%Y', errors='coerce')
MTG = MTG.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MTG rate available
df_2 = pd.merge_asof(df, MTG, left_on='latest_saledate', right_on='observation_date', direction='backward')

# Optional: Remove the duplicate 'date' column if no longer need it
df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'Income', 'population', 'density',
       'MORTGAGE30US'],
      dtype='object')

In [141]:
#merge median list price
MPPSQ['observation_date'] = pd.to_datetime(MPPSQ['observation_date'], format='%d/%m/%Y', errors='coerce')
MPPSQ = MPPSQ.sort_values('observation_date')

# Merge using asof so that each sale date gets the most recent MPPSQ available
df_2 = pd.merge_asof(df_2, MPPSQ, left_on='latest_saledate', right_on='observation_date', direction='backward')

df_2.drop(columns=['observation_date'], inplace=True)
df_2.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasAssociation', 'hasCooling', 'hasGarage', 'hasHeating', 'hasSpa',
       'hasView', 'parkingSpaces', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'latest_saledate', 'numOfPhotos',
       'numOfAccessibilityFeatures', 'numOfAppliances', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWaterfrontFeatures', 'numOfWindowFeatures',
       'numOfCommunityFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfPrimarySchools', 'numOfElementarySchools', 'numOfMiddleSchools',
       'numOfHighSchools', 'avgSchoolDistance', 'avgSchoolRating',
       'avgSchoolSize', 'MedianStudentsPerTeacher', 'numOfBathrooms',
       'numOfBedrooms', 'numOfStories', 'Income', 'population', 'density',
       'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
      dtype='object')

At this point, we have a complete dataset 'df_2' that can be used for further development. 